# Modelos de Cáculo

## CFG

In [ ]:
import nltk
from nltk import CFG

def probar_gramatica():
    # Definir la gramática libre de contexto
    gramatica = CFG.fromstring("""
    S -> A | B
    A -> "0" A "2" | "1" B "2" | ""
    B -> "1" B "2" | ""
    """)

    # Crear el parser para la gramática
    parser = nltk.ChartParser(gramatica)

    # Definir una oración de ejemplo
    oracion = "0122"
    palabras = oracion.split()  # Asegúrate de que las palabras estén separadas

    # Intentar analizar la oración
    print(f"Probando la oración: {oracion}")
    for arbol in parser.parse(palabras):
        print(arbol)



probar_gramatica()


In [ ]:
import nltk
from nltk import CFG
import itertools

def generar_cadenas(gramatica, simbolo_inicial='S', max_longitud=5):
    """
    Genera todas las strings posibles de la gramática hasta una longitud máxima.
    
    :param gramatica: La gramática en formato CFG.
    :param simbolo_inicial: El símbolo inicial (por defecto 'S').
    :param max_longitud: La longitud máxima de las strings generadas.
    :return: Una lista con todas las strings posibles.
    """
    producciones = gramatica.productions()
    
    def es_terminal(simbolo):
        """Verifica si un símbolo es terminal."""
        return simbolo in gramatica._lexical_index
    
    def generar_desde(simbolo, longitud_actual=0, cadena_actual=""):
        """Genera recursivamente las cadenas a partir de un símbolo no terminal."""
        if es_terminal(simbolo):
            cadena_nueva = cadena_actual + simbolo
            if len(cadena_nueva) <= max_longitud:
                return [[simbolo]]
            else:
                return []

        cadenas = []
        for produccion in producciones:
            if produccion.lhs() == simbolo:
                partes = produccion.rhs()
                
                # Generar subcadenas de cada parte
                subcadenas = [generar_desde(s, longitud_actual + 1, cadena_actual) for s in partes]
                
                for combinacion in itertools.product(*subcadenas):
                    cadena_generada = [item for sublista in combinacion for item in sublista]
                    cadena_completa = ''.join(cadena_generada)
                    
                    # Solo añadir cadenas que cumplan con la longitud máxima
                    if len(cadena_completa) <= max_longitud:
                        cadenas.append(cadena_generada)
                        
        return cadenas
    
    # Generar todas las cadenas desde el símbolo inicial
    todas_cadenas = generar_desde(gramatica.start())
    
    # Unir y filtrar por longitud exacta
    return [''.join(cadena) for cadena in todas_cadenas if len(''.join(cadena)) == max_longitud]

# Definir la gramática libre de contexto
gramatica = CFG.fromstring("""
S -> A | B
A -> '0' A '2' | '1' B '2' | ""
B -> '1' B '2' | ""
""")

# Generar cadenas hasta longitud 15
cadenas_generadas = generar_cadenas(gramatica, max_longitud=15)

# Imprimir las cadenas generadas
for cadena in cadenas_generadas:
    ceros = cadena.count('0')
    unos = cadena.count('1')
    doses = cadena.count('2')
    print(f'{ceros} + {unos} =? {doses}', ceros + unos == doses)
    print(f"Cadena generada: '{cadena}'")


## Algoritmos de Markov

In [11]:
#Clase 
import warnings

class algoritmo_de_Markov:
    def __init__(self, reglas, alfabeto =None):
        if alfabeto is None:
            self.alfabeto = [elem for regla in reglas for elem in regla[:2]]
        else:
            self.alfabeto = set(alfabeto)
        self.reglas = reglas
        self._check_reglas()
        pass

    def _check_reglas(self):
        existe_terminal = False
        num_regla = 1
        for patron, remplazo, ind_terminal in self.reglas:
            for i in patron:
                if i not in self.alfabeto:
                    raise ValueError(f"Regla {num_regla}: El símbolo '{i}' no pertenece al alfabeto")
            for j in remplazo:
                if j not in self.alfabeto:
                    raise ValueError(f"Regla {num_regla}: El símbolo '{j}' no pertenece al alfabeto")
            
            if ind_terminal:
                existe_terminal = True
            
            num_regla += 1

        if not existe_terminal:
            warnings.warn("No hay reglas terminales")
        pass

    def _check_string(self, string):
        for i in string:
            if i not in self.alfabeto:
                raise ValueError(f"String: El símbolo '{i}' no pertenece al alfabeto")
        
    def exec_string(self, string, verbose=False):
        self._check_string(string)
        while True:
            aplicada = False  # Bandera para verificar si alguna regla fue aplicada
            for num_regla, (patron, remplazo, ind_terminal) in enumerate(self.reglas, start=1):
                pos = string.find(patron)
                if pos != -1:
                    string = string[:pos] + remplazo + string[pos + len(patron):]
                    aplicada = True
                    if verbose:
                        print(f"Estado string: {string}")
                        print(f'Se aplica la regla {num_regla}: "{patron}" -> "{remplazo}"')
                    if ind_terminal:
                        if verbose:
                            print("Regla terminal aplicada, deteniendo ejecución.")
                        return string  # Si es terminal, termina aquí
                    break  # Reinicia desde la primera regla
            if not aplicada:
                # Si no se aplicó ninguna regla en este ciclo, termina la ejecución
                if verbose:
                    print("No se ha podido aplicar ninguna regla más")
                return string

In [ ]:
#Ejemplo 2
alfabeto = ['a', 'b', 'X', 'Y']
R = [
    ("ab", "X", False),
    ("ba", "Y", True)
]

modelo = algoritmo_de_Markov(R, alfabeto)

print(modelo.alfabeto)
s= 'ababba'
print(s)
print(modelo.exec_string(s, verbose=True))


In [ ]:
#Ejemplo 3

alfabeto = ['a', 'b', '', '0', '1']
R = [
    ('ba', '0', False),
    ('a0', '0', False),
    ('0a', '0', False),
    ('b0', '0', False),
    ('0b', '0', False),
    ('ab', '', False),
    ('a', '0', False),
    ('b', '0', False),
    ('0', '0', True),
    ('', '1', True)
]

pruebas = ['aaabbb', 'aabbb', 'aaa']

modelo = algoritmo_de_Markov(R, alfabeto)

for p in pruebas:
    if modelo.exec_string(p, verbose=True) == '1':
        print(f'La cadena {p} es valida')
    else:
        print(f'La cadena {p} no es valida')

In [ ]:
# Ejemplo 4
import string

alfabeto = list(string.ascii_lowercase)  # Genera ['a', 'b', ..., 'z']
alfabeto.extend(['ñ',' ', ',', '.', '?'])  # Agrega xtras

print(alfabeto)
R = [
    ('esdto', 'esto', False),
    ('qeu', 'que', False),
    ('hoal', 'hola', False),
    ('cmo', 'como', False),
    ('algo', '', True)
]


modelo = algoritmo_de_Markov(reglas=R, alfabeto=alfabeto)

string = 'hoal, cmo estas? esdto qeu te envio tiene errores.'
modelo.exec_string(string, verbose=True)


In [ ]:
# Ejemplo 5
import string

reglas_morse = [
    ("/.-/", "A", False), ("/-.../", "B", False), ("/-.-./", "C", False), ("/-../", "D", False),
    ("/./", "E", False), ("/..-./", "F", False), ("/--./", "G", False), ("/..../", "H", False),
    ("/../", "I", False), ("/.---/", "J", False), ("/-.-/", "K", False), ("/.-../", "L", False),
    ("/--/", "M", False), ("/-./", "N", False), ("/---/", "O", False), ("/.--./", "P", False),
    ("/--.-/", "Q", False), ("/.-./", "R", False), ("/.../", "S", False), ("/-/", "T", False),
    ("/..-/", "U", False), ("/...-/", "V", False), ("/.--/", "W", False), ("/-..-/", "X", False),
    ("/-.--/", "Y", False), ("/--../", "Z", False),
    # Números
    ("/-----/", "0", False), ("/.----/", "1", False), ("/..---/", "2", False), ("/...--/", "3", False),
    ("/....-/", "4", False), ("/...../", "5", False), ("/-..../", "6", False), ("/--.../", "7", False),
    ("/---../", "8", False), ("/----./", "9", False),
    # Espacios entre letras y palabras en Morse
    ("_", " ", False)
]

reglas_morse_ordenadas = sorted(reglas_morse, key=lambda x: len(x[0]), reverse=True)

# Definir el alfabeto que incluye '.' y '-' para representar morse
alfabeto_morse = ['.', '-', ' ', '/', '_']
alfabeto_morse.extend(string.ascii_uppercase)
alfabeto_morse.extend(string.digits)

# Crear una instancia del algoritmo de Markov para decodificar Morse
decodificador_morse = algoritmo_de_Markov(reglas_morse, alfabeto=alfabeto_morse)

# Decodificar un mensaje de ejemplo
mensaje_morse = "/.-/_/-.../_/-.-./"
resultado = decodificador_morse.exec_string(mensaje_morse, verbose=True)
print(f"Mensaje decodificado: {resultado}")


# Pruebas

In [18]:
#Clase 
import warnings

class algoritmo_de_Markov:
    def __init__(self, reglas, alfabeto =None):
        if alfabeto is None:
            self.alfabeto = [elem for regla in reglas for elem in regla[:2]]
        else:
            self.alfabeto = set(alfabeto)
        self.reglas = reglas
        self._check_reglas()
        pass

    def _check_reglas(self):
        existe_terminal = False
        num_regla = 1
        for patron, remplazo, ind_terminal in self.reglas:
            for i in patron:
                if i not in self.alfabeto:
                    raise ValueError(f"Regla {num_regla}: El símbolo '{i}' no pertenece al alfabeto")
            for j in remplazo:
                if j not in self.alfabeto:
                    raise ValueError(f"Regla {num_regla}: El símbolo '{j}' no pertenece al alfabeto")
            
            if ind_terminal:
                existe_terminal = True
            
            num_regla += 1

        if not existe_terminal:
            warnings.warn("No hay reglas terminales")
        pass

    def _check_string(self, string):
        for i in string:
            if i not in self.alfabeto:
                raise ValueError(f"String: El símbolo '{i}' no pertenece al alfabeto")
        
    def exec_string(self, string, verbose=False):
        self._check_string(string)
        while True:
            aplicada = False  # Bandera para verificar si alguna regla fue aplicada
            for num_regla, (patron, remplazo, ind_terminal) in enumerate(self.reglas, start=1):
                pos = string.find(patron)
                if pos != -1:
                    string = string[:pos] + remplazo + string[pos + len(patron):]
                    aplicada = True
                    if verbose:
                        print(f"Estado string: {string}")
                        print(f'Se aplica la regla {num_regla}: "{patron}" -> "{remplazo}"')
                    if ind_terminal:
                        if verbose:
                            print("Regla terminal aplicada, deteniendo ejecución.")
                        return string  # Si es terminal, termina aquí
                    break  # Reinicia desde la primera regla
            if not aplicada:
                # Si no se aplicó ninguna regla en este ciclo, termina la ejecución
                if verbose:
                    print("No se ha podido aplicar ninguna regla más")
                return string

    def gen_strings(self, longitud, input_string="", verbose=False):
        """Genera palabras aplicando reglas hasta que se alcance la longitud deseada."""
        self._check_string(input_string)
        string = input_string

        while len(string) < longitud:
            aplicada = False
            for patron, remplazo, ind_terminal in self.reglas:
                pos = string.find(patron)
                if pos != -1:
                    string = string[:pos] + remplazo + string[pos + len(patron):]
                    aplicada = True
                    if verbose:
                        print(f'Se aplica la regla: "{patron}" -> "{remplazo}"')
                        print(f"Estado string: {string}")
                    if ind_terminal or len(string) >= longitud:
                        if verbose:
                            print("Regla terminal aplicada o longitud alcanzada.")
                        return string[:longitud]  # Corta la palabra a la longitud exacta
                    break  # Reiniciar el ciclo de reglas
            if not aplicada:
                if verbose:
                    print("No se ha podido aplicar ninguna regla más")
                break  # No hay más reglas aplicables

        return string[:longitud]  # Devolver la palabra generada o cortarla a la longitud deseada

# Definir alfabeto y reglas de Markov
alfabeto = "SAB012"
reglas = [
    ("S", "A", False),  # S -> A
    ("S", "B", False),  # S -> B
    ("A", "0A2", False),  # A -> '0' A '2'
    ("A", "1B2", False),  # A -> '1' B '2'
    ("A", "", True),  # A -> ""
    ("B", "1B2", False),  # B -> '1' B '2'
    ("B", "", True)  # B -> "" (terminal)
]


# Crear una instancia del algoritmo de Markov
markov = algoritmo_de_Markov(reglas=reglas, alfabeto=alfabeto)

# Generar cadenas con el algoritmo de Markov
cadenas_markov = markov.gen_strings(input_string="S", longitud=10, verbose=True)



Se aplica la regla: "S" -> "A"
Estado string: A
Se aplica la regla: "A" -> "0A2"
Estado string: 0A2
Se aplica la regla: "A" -> "0A2"
Estado string: 00A22
Se aplica la regla: "A" -> "0A2"
Estado string: 000A222
Se aplica la regla: "A" -> "0A2"
Estado string: 0000A2222
Se aplica la regla: "A" -> "0A2"
Estado string: 00000A22222
Regla terminal aplicada o longitud alcanzada.


In [ ]:
# Definir alfabeto y reglas
alfabeto = "abcX"
reglas = [
    ("a", "bX", False),  # Reemplaza 'a' con 'bX'
    ("X", "cc", True),   # Reemplaza 'X' con 'cc' (regla terminal)
    ("b", "ab", False)   # Reemplaza 'b' con 'ab'
]

# Crear una instancia del algoritmo de Markov
markov = algoritmo_de_Markov(alfabeto, reglas)

# Generar palabras con longitud máxima de 10 caracteres
input_string = "aabb"
palabras_generadas = markov.generar_palabras(input_string, max_iteraciones=10, longitud_max=10, verbose=True)

# Mostrar las palabras generadas
print("\nPalabras generadas:")
for palabra in palabras_generadas:
    print(palabra)

# Plotly

In [19]:
import nltk
from nltk import CFG
import itertools
import plotly.graph_objects as go
from collections import Counter
import numpy as np

def generar_cadenas(gramatica, simbolo_inicial='S', max_longitud=5):
    """
    Genera todas las strings posibles de la gramática hasta una longitud máxima.
    
    :param gramatica: La gramática en formato CFG.
    :param simbolo_inicial: El símbolo inicial (por defecto 'S').
    :param max_longitud: La longitud máxima de las strings generadas.
    :return: Una lista con todas las strings posibles.
    """
    producciones = gramatica.productions()
    
    def es_terminal(simbolo):
        """Verifica si un símbolo es terminal."""
        return isinstance(simbolo, str) and simbolo in gramatica._lexical_index
    
    def generar_desde(simbolo, longitud_actual=0):
        """Genera recursivamente las cadenas a partir de un símbolo no terminal."""
        if es_terminal(simbolo):
            return [[simbolo]]  # Devuelve la cadena terminal como lista

        if longitud_actual >= max_longitud:
            return []  # No generar más si se ha alcanzado la longitud máxima

        cadenas = []
        for produccion in producciones:
            if produccion.lhs() == simbolo:
                partes = produccion.rhs()
                longitud_nueva = longitud_actual + sum(1 for s in partes if es_terminal(s))
                if longitud_nueva > max_longitud:
                    continue  # No continuar si la longitud de la cadena excede el límite
                subcadenas = [generar_desde(s, longitud_actual + 1) for s in partes]
                for combinacion in itertools.product(*subcadenas):
                    cadena_generada = [item for sublista in combinacion for item in sublista]
                    if len(cadena_generada) <= max_longitud:
                        cadenas.append(cadena_generada)
        return cadenas
    
    # Generar todas las cadenas desde el símbolo inicial
    todas_cadenas = generar_desde(gramatica.start())
    cadenas_unicas = set([' '.join(cadena) for cadena in todas_cadenas])
    return list(cadenas_unicas)

def graficar_distribucion_palabras_todas_iteraciones(cadenas_generadas_por_iteracion):
    """
    Grafica una dispersión de las palabras generadas en el espacio tridimensional (ceros, unos, doses) para todas las iteraciones usando Plotly.
    
    :param cadenas_generadas_por_iteracion: Lista de listas, donde cada sublista contiene las cadenas generadas en una iteración.
    """
    fig = go.Figure()

    colores = np.linspace(0, 1, len(cadenas_generadas_por_iteracion))
    
    # Invertir el orden de las iteraciones para plotear desde la última hasta la primera
    for idx in reversed(range(len(cadenas_generadas_por_iteracion))):
        cadenas_generadas = cadenas_generadas_por_iteracion[idx]
        ceros = []
        unos = []
        doses = []

        for cadena in cadenas_generadas:
            ceros.append(cadena.count('0'))
            unos.append(cadena.count('1'))
            doses.append(cadena.count('2'))
        
        fig.add_trace(go.Scatter3d(
            x=ceros,
            y=unos,
            z=doses,
            mode='markers',
            marker=dict(
                size=5,
                color=colores[idx],  # Color diferente para cada iteración
                colorscale='Viridis',
                opacity=0.8
            ),
            name=f'Iteración {idx + 1}'
        ))
    
    fig.update_layout(
        scene=dict(
            xaxis_title='Cantidad de 0s',
            yaxis_title='Cantidad de 1s',
            zaxis_title='Cantidad de 2s'
        ),
        title='Dispersión de la Distribución de las Palabras Generadas en Todas las Iteraciones'
    )
    fig.show(renderer='browser')  # Cambiar el renderer a 'browser' para evitar problemas de visualización en algunos entornos

# Definir la gramática libre de contexto
gramatica = CFG.fromstring("""
S -> A | B
A -> '0' A '2' | '1' B '2' | ""
B -> '1' B '2' | ""
""")

# Generar y acumular las cadenas en cada iteración
generar_hasta = 50
cadenas_generadas_por_iteracion = []

for n in range(1, generar_hasta + 1):
    cadenas_generadas = generar_cadenas(gramatica, max_longitud=n)
    cadenas_generadas_por_iteracion.append(cadenas_generadas)
    
    # Imprimir las cadenas generadas
    print(f"Iteración {n}", cadenas_generadas)
    # for cadena in cadenas_generadas:
    #     ceros = cadena.count('0')
    #     unos = cadena.count('1')
    #     doses = cadena.count('2')
    #     print(f'{ceros} + {unos} =? {doses}', ceros + unos == doses)
    #     print(f"Cadena generada: '{cadena}'")

# Graficar la distribución de las letras en las palabras generadas para todas las iteraciones
graficar_distribucion_palabras_todas_iteraciones(cadenas_generadas_por_iteracion)


Iteración 1 []
Iteración 2 ['']
Iteración 3 ['1  2', '0  2', '']
Iteración 4 ['1  2', '0  2', '']
Iteración 5 ['0 0  2 2', '0 1  2 2', '', '1 1  2 2', '1  2', '0  2']
Iteración 6 ['0 0  2 2', '0 1  2 2', '', '1 1  2 2', '1  2', '0  2']
Iteración 7 ['0 0  2 2', '0 1  2 2', '1 1 1  2 2 2', '', '0 1 1  2 2 2', '1 1  2 2', '1  2', '0 0 1  2 2 2', '0  2', '0 0 0  2 2 2']
Iteración 8 ['0 0  2 2', '0 1  2 2', '1 1 1  2 2 2', '', '0 1 1  2 2 2', '1 1  2 2', '1  2', '0 0 1  2 2 2', '0  2', '0 0 0  2 2 2']
Iteración 9 ['0 0  2 2', '0 0 1 1  2 2 2 2', '0 1  2 2', '1 1 1  2 2 2', '', '0 1 1 1  2 2 2 2', '1 1 1 1  2 2 2 2', '0 1 1  2 2 2', '1 1  2 2', '0 0 0 1  2 2 2 2', '1  2', '0 0 0 0  2 2 2 2', '0 0 1  2 2 2', '0  2', '0 0 0  2 2 2']
Iteración 10 ['0 0  2 2', '0 0 1 1  2 2 2 2', '0 1  2 2', '1 1 1  2 2 2', '', '0 1 1 1  2 2 2 2', '1 1 1 1  2 2 2 2', '0 1 1  2 2 2', '1 1  2 2', '0 0 0 1  2 2 2 2', '1  2', '0 0 0 0  2 2 2 2', '0 0 1  2 2 2', '0  2', '0 0 0  2 2 2']
Iteración 11 ['0 0  2 2', '', '